In [1]:
#!pip install tensorflow-gpu
# Downgrade 
#smart_open to 1.10.0 -> https://github.com/RaRe-Technologies/smart_open/issues/475
# python -m pip install -U smart_open==1.10.0

In [2]:
import os
%load_ext tensorboard

In [3]:
###########################################
# How to make deterministic experiments?
###########################################

# Main Sources:
    # 1) https://github.com/NVIDIA/tensorflow-determinism
    # 2) https://pypi.org/project/tensorflow-determinism/#description
            # There are currently two main ways to access GPU-deterministic functionality in TensorFlow for most
            # deep learning applications. 
            # 2.1) The first way is to use an NVIDIA NGC TensorFlow container. - https://www.nvidia.com/en-us/gpu-cloud/containers/
            # 2.2. The second way is to use version 1.14, 1.15, or 2.0 of stock TensorFlow with GPU support, 
            #      plus the application of a patch supplied in this repo.

# # # Ensure Deterministic behaviour
import random
import numpy as np
import tensorflow as tf
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Now using tensorflow 2.1.0, so no need to patch
# from tfdeterminism import patch
#patch()

seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
############################################

In [4]:
# utils
from distutils.version import LooseVersion
from tqdm import tqdm_notebook
from datetime import datetime
from time import time

import warnings
import pickle
import gc
import sys
from json import dumps
import itertools
import re

# Data
import pandas as pd
import spacy

# Viz
import matplotlib.pyplot as plt

# Machine Learning
import tensorflow.keras.backend as K
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Bidirectional, LSTM, Flatten, Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks.callbacks import EarlyStopping
#from keras.callbacks import EarlyStopping, TensorBoard
#from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHead, MultiHeadAttention

from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

# NLP Models
from gensim.models import Word2Vec, KeyedVectors
#w2v_models_path = 'C:/Users/arthu/Desktop/22032020 - Experimentos/05. Organizado/02. Notebooks/models/'
w2v_models_path = 'D:/Mestrado/Dissertação/07 .Dissertação Final/02. Experimentos/02. Word Embbedings/'

Using TensorFlow backend.


In [5]:
# METRICS
# def f1_score(y_true, y_pred):

#     # Count positive samples.
#     c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

# #     # If there are no true samples, fix the F1 score at 0.
# #     if c3 == 0:
# #         return 0

#     # How many selected items are relevant?
#     precision = c1 / c2

#     # How many relevant items are selected?
#     recall = c1 / c3

#     # Calculate f1_score
#     f1_score = 2 * (precision * recall) / (precision + recall)
#     return f1_score

# def recall(y_true, y_pred):

#     # Count positive samples.
#     c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

# #     # If there are no true samples, fix the F1 score at 0.
# #     if c3 == 0:
# #         return 0

#     # How many relevant items are selected?
#     recall = c1 / c3
    
#     return recall


# def precision(y_true, y_pred):

#     # Count positive samples.
#     c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))

#     # How many selected items are relevant?
#     precision = c1 / c2

#     return precision

# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
print('tf version: ' + tf.__version__)

tf version: 2.1.0-rc2


In [7]:
#Variables
current_exp = 'Atendimento-Unbalanced-Binary'
if 'Binary' in current_exp:
    binary = True
else:
    binary = False
    
base_path = 'C:/Users/arthu/Desktop/22032020 - Experimentos/05. Organizado/03. Datasets/' + current_exp
save_path = 'output'

sentence = 'resp-text'
label = 'Atendimento'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train[label].value_counts())
print(y_test[label].value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       '58home-response', '59money-response', '60relig-response',
       '61death-response', '62assent-response', '63nonfl-response',
       '64filler-response', 'Clareza', 'Atendimento', 'tempo_resposta'],
      dtype='object', length=134)
(31586, 134)
2    21682
0     9904
Name: Atendimento, dtype: int64
2    9338
0    4200
Name: Atendimento, dtype: int64


In [8]:
# Keep only text columns
X_train.drop(columns=X_train.columns[3:], inplace=True)
X_test.drop(columns=X_train.columns[3:], inplace=True)

In [9]:
###################################################
X_train['sentence'] = X_train[sentence]
X_test['sentence'] = X_test[sentence]

y_train['label'] = y_train[label]
y_test['label'] = y_test[label]

##################################################################
# CUT DATAFRAME
# factor = 10000
# df = pd.concat([df[df.label=='1'][0:factor], df[df.label=='0'][0:factor]])
##################################################################

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(X_train.shape[0]))

# Report the classes balance.
print('Classes distribuition: \n')
print(y_train[label].value_counts())

# Display 10 random rows from the data.
X_train.sample(10)

Number of training sentences: 31,586

Classes distribuition: 

2    21682
0     9904
Name: Atendimento, dtype: int64


,pid,req-text,resp-text,sentence
27971,519005,"Prezados, solicito informações sobre o FNO des...",Prezado Senhor : Obrigado por utilizar esse ca...,Prezado Senhor : Obrigado por utilizar esse ca...
4148,557020,Quantos servidores ocupantes do cargo de Deleg...,"Caro Requerente , Em resposta ao seu pedido de...","Caro Requerente , Em resposta ao seu pedido de..."
2527,367364,Há código de vaga para Psicólogo no HU/UFGD?,Encaminhamos vossa solicitação para a Divisão...,Encaminhamos vossa solicitação para a Divisão...
29114,452095,"Prezados, gostaria de esclarecimentos sobre a ...","Prezado Sr . Edson , Em resposta a sua solicit...","Prezado Sr . Edson , Em resposta a sua solicit..."
2388,489220,No site dados.gov.br têm sido disponibilizados...,"Senhora Carolina, O Serviço de Informações ao ...","Senhora Carolina, O Serviço de Informações ao ..."
22896,555233,"O Requerente, AFT/MTE aposentado, SIAPE 025372...","Prezado a Senhor a , Em resposta ao seu pedido...","Prezado a Senhor a , Em resposta ao seu pedido..."
23246,405393,Fiz um pedido anterior de informações sobre co...,"Prezado , Em resposta ao seu pedido informamos...","Prezado , Em resposta ao seu pedido informamos..."
16050,468913,"Observação: As perguntas a seguir, ficam mais ...","Prezado , Segue resposta . Informamos que cas...","Prezado , Segue resposta . Informamos que cas..."
6778,441341,Gostaria de saber a quantidade de códigos de v...,"Prezado , Informo que não há código de vaga di...","Prezado , Informo que não há código de vaga di..."
1723,512226,Solicito da CAIXA ECONÔMICA FEDERAL cópia do p...,"Prezado a Cidadão ã , 1 . Conforme solicitação...","Prezado a Cidadão ã , 1 . Conforme solicitação..."


In [10]:
# Looking lengths
lengths = [X_train.sentence.apply(lambda x: len(x.split(' ')))]
perc =[.25, .50, .75, .80, .85, .90, .91, .92, .93, .94, .95, .96, .97, .98, .99] 
lengths[0].describe(percentiles = perc)

count    31586.000000
mean       158.355379
std        164.789252
min          1.000000
25%         52.000000
50%        110.000000
75%        207.000000
80%        239.000000
85%        279.000000
90%        334.000000
91%        352.000000
92%        371.000000
93%        395.000000
94%        425.000000
95%        453.750000
96%        493.000000
97%        549.000000
98%        670.000000
99%        862.000000
max       1885.000000
Name: sentence, dtype: float64

# w2v Model for Embedding Layer

In [11]:
#w2v_cbow_esic_model=KeyedVectors.load(os.path.join(w2v_models_path,'word2vec_sg_hs_DetalhamentoSolicitacao_all_sentences_128.model'))
w2v_cbow_nilc_model=KeyedVectors.load_word2vec_format(os.path.join(w2v_models_path,'cbow_s300.txt'))

KeyboardInterrupt: 

In [ ]:
pretrained_weights = w2v_cbow_nilc_model.wv.syn0
print(pretrained_weights.shape)
max_num_words = pretrained_weights.shape[0]
embed_size = pretrained_weights.shape[1]

# Data Prep

In [ ]:
max_length=128

# Define tokenizer and fit train data
t = Tokenizer(num_words=max_num_words)
t.fit_on_texts(X_train['sentence'].append(X_test['sentence']))
word_index = t.word_index
vocab_size = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))
    
def get_seqs(text):    
    sequences = t.texts_to_sequences(text)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

In [ ]:
# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    #if binary:
    #    return y_train_enc, y_test_enc
    #else:
    return pd.get_dummies(y_train_enc), pd.get_dummies(y_test_enc)

In [ ]:
# X and Y
label_train, label_test = prepare_targets(y_train.label.values, y_test.label.values)
num_labels = len(set(label_train))
input_train = get_seqs(X_train.sentence)
input_test = get_seqs(X_test.sentence)

# Modeling

In [ ]:
embedding_matrix = np.zeros((vocab_size, embed_size))
for word, i in t.word_index.items():
    try:
        embedding_vector = w2v_cbow_nilc_model.wv.__getitem__(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    # Words not in vocab -> Frequency less than 5 word
    except KeyError as e:
        print(e)

In [ ]:
# Define model
def train_model(input_train, input_test, label_train, label_test,
                lstm_size=128, dropout=0.2, rec_dropout=0.2, lr=0.005, epochs=50, att_heads=4, max_length=128, 
                vocab_size=None, embed_size=None, emb_trainable=False, batch=128, early_stopping=5,
                save_dir="D:/resultados/checkpoins_solicitacao_keras_mh_att/", best_predefined_f1=0.390):

    # Time now
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Log
    log_dir = save_dir + now
    
    # Model Saver    
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    log_file = open(os.path.join(log_dir,"log.txt"), mode="a")
    
    # Save Params
    params = {
        'lstm_size': lstm_size,
        'dropout': dropout,
        'rec_dropout': rec_dropout,
        'lr': lr,
        'epochs': epochs,
        'att_heads': att_heads,
        'max_length': max_length,
        'vocab_size': vocab_size,
        'embed_size': embed_size,
        'emb_trainable': emb_trainable,
        'batch': batch,
        'early_stopping': early_stopping,
        'log_dir': log_dir,
        'best_predefined_f1': best_predefined_f1}
    
    # Saving Parameters
    with open(os.path.join(log_dir, 'params.txt'),'a') as f:
        f.write('\n\n' + ('#'*60))
        f.write('\nParameters:\n')
        f.write('now: ' + str(now))
        f.write('\n' + dumps(params) + '\n')
    
    # input
    inp = Input(shape=(max_length, ))

    # Embedding layer - https://keras.io/layers/embeddings/
    embedding_layer = Embedding(vocab_size,
                                embed_size,                                
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=emb_trainable,
                                name='Embedding')(inp)

    # Bidirectional Layer
    bilstm_layer = Bidirectional(LSTM(
                        units=lstm_size,
                        return_sequences=True,
                        dropout=dropout,
                        recurrent_dropout=rec_dropout,
                        name='LSTM'))(embedding_layer)    

    # MultiHead-Attention Layer
    #https://pypi.org/project/keras-multi-head/
    multiHead_att_layer = MultiHeadAttention(head_num=att_heads, name='Multi-Head-Attention')(bilstm_layer)

    dropout_intermed_layer = Dropout(0.5)(multiHead_att_layer)

    # # Flatten
    flatten_layer = Flatten(name='Flatten')(dropout_intermed_layer)

    dense_intermed_layer = Dense(128, activation='relu')(flatten_layer)
    dropout_intermed_2_layer = Dropout(0.5)(dense_intermed_layer)

    # # # # Dense Layer
    #if binary:
    #    dense_layer = Dense(1, activation='sigmoid')(flatten_layer)    
    #else:
    dense_layer = Dense(num_labels, activation='softmax')(dropout_intermed_2_layer)    
    
    model = Model(inputs=inp, outputs=dense_layer)
    # model.summary()
    
    # Compile
    model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy', precision_m, recall_m, f1_m])
    
    # callbacks
    es_callback = EarlyStopping(monitor='val_loss', patience=early_stopping, verbose=1, mode='min')
    
    # class weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(label_train), np.argmax(label_train.values, axis=1))

    #https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

    # Scaling by total/2 helps keep the loss to a similar magnitude.
    # The sum of the weights of all examples stays the same.
    #weight_for_0 = (1 / (len(label_train)-sum(label_train)))*(len(label_train))/2.0 
    #weight_for_1 = (1 / sum(label_train))*(len(label_train))/2.0

    #class_weights = {0: weight_for_0, 1: weight_for_1}
    
    # Fitting Model
    model.fit(input_train,
              label_train,
              epochs=epochs,
              batch_size=batch,
              validation_data=(input_test, label_test),
              verbose=0,
              class_weight=class_weights,
              callbacks=[es_callback])
    
    # PLOT LOSS
    plt.title('Loss')
    plt.plot(model.history.history['loss'], label='train')
    plt.plot(model.history.history['val_loss'], label='test')
    plt.legend()
    #plt.show();
    plt.savefig(os.path.join(log_dir,'Loss.png'))
    plt.close()
    
    # Classification
    y_pred = model.predict(input_test, batch_size=batch, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    
    #if not binary:
    label_test = np.argmax(label_test.values, axis=1)
    
    # Metrics
    f1 = f1_score(label_test, y_pred_bool, average='weighted')
    print(f"Best Test F1-Score: {f1:.3f}")
    
    print("#"*60 + '\n', file=log_file)
    print(classification_report(label_test, y_pred_bool), file=log_file)
    print("#"*60+ '\n', file=log_file)
    
    # Flush log file
    log_file.flush()
    log_file.close()
    
    # Save final result
    with open(os.path.join(log_dir[:-16], 'output.txt'),'a') as f:
        f.write('\n\n')
        f.write(log_dir)
        f.write('\n')
        f.write(f"Best Test F1-Score: {f1:.3f}")
        
    # save model and architecture to single file
    if f1 > best_predefined_f1:
        model.save(os.path.join(log_dir, "model.h5"))
        print("Saved model to disk")

In [ ]:
# Model Params
lstm_size_list = [256, 512]
dropout_list = [0.5]
rec_dropout_list = [0.5]
lr_list = [1e-3, 5e-4, 1e-4, 5e-5, 5e-6]

all_params = [lstm_size_list] + [dropout_list] + [rec_dropout_list] + [lr_list]

for each in itertools.product(*all_params):    
    lstm_size, dropout, rec_dropout, lr = each
    
    # Params
    print('lstm_size: ' + str(lstm_size))
    print('\tdropout: ' + str(dropout))
    print('\trec_dropout: ' + str(rec_dropout))
    print('\tlr: ' + str(lr))
    
    # train
    train_model(input_train, input_test, label_train, label_test, lstm_size, dropout, rec_dropout, lr, epochs=50,
                att_heads=4, max_length=max_length, vocab_size=vocab_size, embed_size=embed_size, emb_trainable=False, 
                batch=128, early_stopping=5,
                save_dir="D:/Outputs_Mestrado/resultados_Atendimento/checkpoints_resposta_keras_BiLSTM_mha_binary_unbalanced/",
                best_predefined_f1=0.72)